## GSE 512 Macroeconomic Analysis Final Exam
### Michael Chan

#### I deleted problem 1 since I don't want to publish any comprehensive exam style questions. Problem 2 is a dynamic programming problem.

## Problem 2

1. Include at least four states.  Let one of the states be that the student gets stuck in jail. (5 points)
2. Let the control set include: watch TV; work out; study; go to the bar; and others as you wish. (5 points)
  
  
These were generated using the interactive code from 11 February 2021 notebook. Left that out to reduce clutter.

In [19]:
state_names = ['In Jail', 'Feel Sick', 'Feel Normal', 'Feel Great']
control_names = ['Watch TV','Work Out','Study','go to the bar','rob a bank']
reward = np.array([[-50,-70,-60,-100,200],[-5,-15,-2,-30,10],[5,5,4,30,100],[10,15,12,20,120]])

In [20]:
#some paramaters to work with
beta=0.97

In [31]:
transition #made with the interactive code to create transition.

[array([[0.9, 0. , 0.1, 0. ],
        [0. , 0.6, 0.4, 0. ],
        [0. , 0.1, 0.7, 0.2],
        [0. , 0.1, 0.7, 0.2]]),
 array([[0.85, 0.  , 0.15, 0.  ],
        [0.05, 0.55, 0.4 , 0.  ],
        [0.1 , 0.2 , 0.6 , 0.1 ],
        [0.2 , 0.1 , 0.5 , 0.2 ]]),
 array([[0.85, 0.  , 0.15, 0.  ],
        [0.  , 0.4 , 0.6 , 0.  ],
        [0.  , 0.2 , 0.7 , 0.1 ],
        [0.  , 0.1 , 0.8 , 0.1 ]]),
 array([[1.  , 0.  , 0.  , 0.  ],
        [0.2 , 0.7 , 0.1 , 0.  ],
        [0.15, 0.35, 0.5 , 0.  ],
        [0.1 , 0.2 , 0.6 , 0.1 ]]),
 array([[1.  , 0.  , 0.  , 0.  ],
        [0.7 , 0.1 , 0.2 , 0.  ],
        [0.6 , 0.1 , 0.  , 0.3 ],
        [0.45, 0.15, 0.2 , 0.2 ]])]

In [32]:
def bellman(J, R, M, beta):
    """
    J current value function,
    R reward function,
    M transition rule, 
    beta discount factor
    """
    n = R.shape[0]  #number of states
    k = R.shape[1]  #how many controls
    if len(M) != k :
        print("There is not the proper number of transition matrices.")
        return
    for i in range(k):
        if M[i].shape != (n,n) :
            print('Markov matrix', i, 'has the wrong size.')
            return
        if not np.allclose(sum(M[i].T),1):
            print('Markov matrix', i, 'is incorrect.')
            
    Next = np.empty_like(R) # create an empty n x k matrix
    
    for x in range(n):
        for y in range(k):
            Next[x,y] = np.dot(M[y][x],J)  #the expected value of next period's value function
                                            #given we're in state x and chose control y   
    
    next_J = np.empty_like(J)  #value function for the next iteration
    
    for x in range(n):
        next_J[x] = np.max(R[x, :] + beta*Next[x,:])  #the heart of the Bellman recursion
        # the maximum is taken over all the columns, for a fixed state x
        # each column is a control, of course
    return next_J

In [33]:
def compute_value(R,M,beta,show):  #R is reward function, M transition rule, beta is discount factor
    """
    R is the reward
    M is the transition rule
    beta is the discount factor
    show is a logical variable toggles between printing the iterations
    """
    n = R.shape[0]  # number of states
    J = np.zeros(n)      # Initial guess
    next_J = np.empty(n)  # Stores updated guess
    max_iter = 500  #in case you are stuck in an infinite loop
    i = 0   # my counter for iterations

    while i < max_iter:
        next_J = bellman(J, R, M, beta)  # give it J and spitout next iterate of J
        if show == True:
            print(next_J)
        if np.allclose(next_J, J):  #check if the value functions are squeezing together
            break
        else:
            J[:] = next_J   # Copy contents of next_J to J
            i += 1

    return(J)

In [38]:
J = compute_value(reward,transition,beta,False)

In [37]:
def optimal_policy(J, R, M, beta, state_names, control_names):
    """
    J is the true value function,
    R is reward function, 
    M transition rule
    beta is discount factor
    state_names is a list of the states
    control_names is a list of the controls
    """
    n = R.shape[0]  # number of states
    k = R.shape[1]   #number of controls
    Next = np.empty_like(R)  #create an empty array
    for x in range(n):
        for y in range(k):
            Next[x,y] = np.dot(M[y][x],J) #compute the true Bellman recursion matrix
    policy=np.empty_like(J)  #beginning of the optimal policy function
    for x in range(n):        
        policy[x] = np.argmax(R[x, :] + beta*Next[x,:])  #true Bellman recursion
                              #arg.max finds the k that maximizes for row x
        print('The optimal policy in state', state_names[int(x)], 'is to choose action', control_names[int(policy[x])])
    return(policy)

In [39]:
optimal_policy(J,reward,transition,beta,state_names,control_names)

The optimal policy in state In Jail is to choose action rob a bank
The optimal policy in state Feel Sick is to choose action rob a bank
The optimal policy in state Feel Normal is to choose action rob a bank
The optimal policy in state Feel Great is to choose action rob a bank


array([4., 4., 4., 4.])

This answer made me laugh. I'm going to change some of the rewards.

In [52]:
reward2 = np.array([[-50,-70,-60,-100,0],[-5,-15,-2,-30,30],[5,5,4,30,50],[10,15,12,20,30]])
J = compute_value(reward2,transition,beta,False)

In [53]:
policy = optimal_policy(J,reward2,transition,beta,state_names,control_names)
policy

The optimal policy in state In Jail is to choose action rob a bank
The optimal policy in state Feel Sick is to choose action Study
The optimal policy in state Feel Normal is to choose action Watch TV
The optimal policy in state Feel Great is to choose action Study


array([4., 2., 0., 2.])

So looking at these, they make a bit more sense. The first set of optimal policy choices was to always rob a bank. The reward was worth it. However, The change in rewards (maybe I suddenly gained a conscience) with a reduction in reward meant that it was no longer the optimal policy in all states. The only state where I would make the attempt to rob a bank was when I was already in jail. I never worked out. 

In [56]:
def ergodic(m):
    """
    This code finds the ergodic distribution of a Markov matrix
    It has a couple of important updates.
    np.allclose() is a useful function 
    The calculation of the eigenvalues was subject to rounding error.
    Now I use the left nullspace of m instead
    """
    n=m.shape[0] # number of rows
    T = 1000 # useful constant
    if (m.shape[0] != m.shape[1]): #test for a square matrix
        return('Matrix is not square') 
    if (not np.allclose(m@np.ones(n),np.ones(n))): #test row sums 
        return('This is not a Markov matrix')
    if( not ((m >= 0).all() and (m <= 1).all())): #test probabilities
        return('This is not a Markov matrix')
    m_long = np.linalg.matrix_power(m,T)
    if( not ((m_long > 0).all() and (m_long < 1).all())): #crude test for aperiodic and irreducible
        return('This matrix may not be both aperiodic and irreducible')
    m0=null_space(m.T-np.identity(n)) #need the nullspace of the rows.   That's why I transpose
    return(m0/sum(m0)) # make it into a probability distribution
def ergodic_states(policy,transition,state_names):
    """
    policy is the policy function,
    transition is the transition rule
    state_names is a list of the states
    returns the Markov matrix for the system
    and also the ergodic distribution of the states
    """
    n = policy.shape[0]  # number of states
    M_optimal = np.zeros((n,n)) # zero out the array
    for i in range(n):
        M_optimal[i] = transition[int(policy[i])][i]  #pick the row corresponding to the optimal policy in state i
    ergodic_dist = ergodic(M_optimal)    
    print('The states are', state_names)
    print('Their ergodic distribution is', ergodic_dist)
    return(M_optimal,ergodic_dist)

In [57]:
ergodic_states(policy,transition,state_names)

The states are ['In Jail', 'Feel Sick', 'Feel Normal', 'Feel Great']
Their ergodic distribution is This matrix may not be both aperiodic and irreducible


(array([[1. , 0. , 0. , 0. ],
        [0. , 0.4, 0.6, 0. ],
        [0. , 0.1, 0.7, 0.2],
        [0. , 0.1, 0.8, 0.1]]),
 'This matrix may not be both aperiodic and irreducible')

This is funny looking, but it looks like if I'm in jail I'm going to be in jail forever. I guess that's because the optimal policy is telling me I should try robbing a bank while I'm in jail. As difficult as that sounds, the transition rule that I made up arbitrarily has me going back to jail for sure if I try robbing a bank while I'm in jail. It's a silly scenario, but it all makes sense. The other states don't have me trying to do anything that puts me in jail (which makes sense since being in jail is forever here). 